# Notebook 0: Data Ingestion for Toyota Specifications

## Purpose

This notebook ingests Toyota specification PDFs and creates **hybrid chunks** that support:

1. **Semantic Search**: Natural language queries like "comfortable family sedan"
2. **Metadata Filtering**: Precise filters like `{"model": "Camry", "mpg_combined": {">":30}}`

## Why Hybrid Chunks?

Notebook1's planning concepts require BOTH capabilities:
- **Multi-query variants** need semantic text ("spacious", "reliable", "efficient")
- **Subquery filters** need structured KeySpec metadata

## Strategy

Each chunk contains:
- **Full PDF text** (~2,843 chars avg) for semantic search
- **Structured specifications** for readability  
- **KeySpec metadata** for precise filtering

Total chunk size: ~3,500 chars (~875 tokens) - well within text-embedding-005 limits!

## Section 1: Package Installation

In my case I have installed all the dependencies already following these instructions:

```shell
# Create Python virtual environment
python3.11 -m venv .venv

# Activate newly created Python virtual environment
source .venv/bin/activate

# Install all the dependencies using `requirements.txt`
pip install -r requirements.txt
```

In [1]:
%pip -q install google-cloud-aiplatform vertexai


[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip -q install chromadb pypdf


[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip -q install langchain-community langchain-google-vertexai langchain-core

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vertexai 1.71.1 requires google-cloud-aiplatform[all]==1.71.1, but you have google-cloud-aiplatform 1.126.0 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Section 2: Environment Configuration

Configuration matches Notebook1 exactly to ensure compatibility.

In [4]:
PROJECT_ID = "agentapps-473813"
REGION = "us-central1"

In [5]:
import os

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

# Paths
DATA_DIR = "../data/car-specs/toyota-specs"
PERSIST_DIR = "../data/chroma"
COLLECTION_NAME = "toyota_specs"

# Models
EMBED_MODEL_ID = "text-embedding-005"
EMBED_OUTPUT_DIM = 256
LLAMA_MODEL_ID = "meta/llama-3.3-70b-instruct-maas"

print("Configuration:")
print(f"  DATA_DIR: {DATA_DIR}")
print(f"  PERSIST_DIR: {PERSIST_DIR}")
print(f"  COLLECTION: {COLLECTION_NAME}")
print(f"  EMBED_MODEL: {EMBED_MODEL_ID}")
print(f"  LLM_MODEL: {LLAMA_MODEL_ID}")

Configuration:
  DATA_DIR: ../data/car-specs/toyota-specs
  PERSIST_DIR: ../data/chroma
  COLLECTION: toyota_specs
  EMBED_MODEL: text-embedding-005
  LLM_MODEL: meta/llama-3.3-70b-instruct-maas


## Section 3: Initialize Vertex AI

In [6]:
from google.cloud import aiplatform
import vertexai

aiplatform.init(project=PROJECT_ID, location=REGION)
vertexai.init(project=PROJECT_ID, location=REGION)

print("✓ Vertex AI initialized")

✓ Vertex AI initialized


/Users/itversity/Projects/Internal/aicohort-content/.venv/lib/python3.11/site-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


## Section 4: Schema Definition (KeySpec)

### Critical: Metadata Alignment

This `KeySpec` schema **MUST match Notebook1 exactly** (lines 434-446).

**Why?**
- Notebook1's planning generates filters like `{"mpg_combined": {">":30}}`
- These filters will fail if field names don't match!

### Field Support Matrix

| Field | Semantic Search | Metadata Filter | Example Query |
|-------|----------------|-----------------|---------------|
| model | ✅ (via text) | ✅ | "Show me Camry options" |
| trim | ✅ (via text) | ✅ | "TRD Pro trim" |
| mpg_combined | ❌ | ✅ | "MPG > 30" |
| ev_only_range_mi | ❌ | ✅ | "EV range > 40 miles" |
| towing_max_lbs | ❌ | ✅ | "Can tow 6000 lbs" |
| seats | ❌ | ✅ | "Seats >= 7" |
| drivetrain | ✅ (via text) | ✅ | "AWD vehicles" |


In [7]:
from typing import List, Optional, Dict, Any
from pydantic import BaseModel, Field

class KeySpec(BaseModel):
    """
    Canonical metadata schema for Toyota specs.
    MUST match Notebook1's KeySpec (lines 434-446) exactly!
    """
    model: str  # REQUIRED: e.g., "Camry", "Prius", "RAV4"
    trim: Optional[str] = None  # e.g., "LE", "XLE", "TRD Pro"
    mpg_city: Optional[float] = None
    mpg_hwy: Optional[float] = None
    mpg_combined: Optional[float] = None
    ev_only_range_mi: Optional[float] = None
    total_range_mi: Optional[float] = None
    towing_max_lbs: Optional[float] = None
    seats: Optional[int] = None
    drivetrain: Optional[str] = None  # e.g., "AWD", "FWD", "4WD"
    starting_price_mentions: Optional[List[str]] = None

print("✓ KeySpec schema defined")


✓ KeySpec schema defined


## Section 5: PDF Extraction Functions

### PDF Size Analysis

Our PDFs are small:
- **Average**: 2,843 characters (~711 tokens)
- **Largest**: 3,310 characters (Prius)
- **Embedding limit**: ~3,072 tokens (~12,000 chars)

**Conclusion**: We can safely include full PDF text in each chunk!


In [8]:
from pathlib import Path
from pypdf import PdfReader

def list_pdf_files(data_dir: str) -> List[Path]:
    """List all PDF files in directory."""
    p = Path(data_dir)
    if not p.exists():
        raise FileNotFoundError(f"Directory not found: {data_dir}")
    files = sorted(list(p.glob("*.pdf")))
    if not files:
        raise FileNotFoundError(f"No PDFs found in {data_dir}")
    return files

def extract_text_from_pdf(path: Path) -> str:
    """
    Extract full text from PDF.
    NO truncation - our PDFs are small enough!
    """
    reader = PdfReader(str(path))
    chunks = []
    for pg in reader.pages:
        try:
            text = pg.extract_text() or ""
        except Exception:
            text = ""
        if text:
            chunks.append(text)
    return "\n".join(chunks)

print("✓ PDF extraction functions defined")


✓ PDF extraction functions defined


In [9]:
# Verify PDFs exist
pdf_files = list_pdf_files(DATA_DIR)
print(f"Found {len(pdf_files)} PDFs:")
for pdf in pdf_files:
    print(f"  - {pdf.name}")


Found 8 PDFs:
  - Introduction_to_Toyota_Car_Sales.pdf
  - Toyota_Camry_Specifications.pdf
  - Toyota_Corolla_Specifications.pdf
  - Toyota_Highlander_Specifications.pdf
  - Toyota_Prius_Specifications.pdf
  - Toyota_RAV4_Specifications.pdf
  - Toyota_Tacoma_Specifications.pdf
  - Toyota_bZ4X_Specifications.pdf


## Section 6: LLM Setup for Spec Extraction

We use the same Llama model as Notebook1 to extract structured KeySpec data from PDFs.


In [10]:
from langchain_google_vertexai.model_garden_maas.llama import VertexModelGardenLlama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# Initialize LLM
chat_llm = VertexModelGardenLlama(
    model=LLAMA_MODEL_ID,
    project=PROJECT_ID,
    location=REGION,
)

print(f"✓ LLM initialized: {LLAMA_MODEL_ID}")


✓ LLM initialized: meta/llama-3.3-70b-instruct-maas


In [11]:
# Setup extraction chain
extraction_parser = JsonOutputParser(pydantic_object=KeySpec)

extraction_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a strict JSON generator. Return ONLY valid JSON."),
    ("human", """Extract ALL model/trim configurations from this Toyota PDF.
Return a JSON array of objects matching this schema: {schema}

Extract:
- model (required): e.g., "Camry", "Prius", "RAV4"
- trim: e.g., "LE", "XLE", "TRD Pro", "Hybrid"
- mpg_city, mpg_hwy, mpg_combined (if mentioned)
- ev_only_range_mi, total_range_mi (for hybrids/EVs)
- towing_max_lbs (if mentioned)
- seats (if mentioned)
- drivetrain: "FWD", "AWD", or "4WD"
- starting_price_mentions: list of price strings mentioned

PDF: {filename}
TEXT:
{doc_text}
"""),
])

print("✓ Extraction prompt configured")


✓ Extraction prompt configured


## Section 7: Spec Extraction Function

This function extracts structured KeySpec data AND returns the original text.


In [12]:
def extract_specs_from_pdf_llm(pdf_path: Path, llm) -> tuple[List[KeySpec], str]:
    """
    Extract structured specs AND original text from PDF.
    
    Returns:
        (specs_list, original_text)
        - specs_list: List of KeySpec objects (one per trim)
        - original_text: Full PDF text for semantic search
    """
    # Extract full text
    text = extract_text_from_pdf(pdf_path)
    
    # Call LLM to extract structured data
    chain = extraction_prompt | llm | extraction_parser
    result = chain.invoke({
        "schema": KeySpec.model_json_schema(),
        "filename": pdf_path.name,
        "doc_text": text,
    })
    
    # Normalize to list
    if isinstance(result, dict):
        result = [result]
    
    # Convert to KeySpec objects
    specs = [KeySpec.model_validate(item) for item in result]
    
    # Return BOTH specs and original text
    return specs, text

print("✓ Extraction function defined")


✓ Extraction function defined


## Section 8: Hybrid Chunking Function

### Why Hybrid Chunks?

**Problem with metadata-only chunks:**
```
Content: "Model: Camry\nTrim: LE\nDrivetrain: FWD"
```
❌ Won't match "comfortable family sedan"  
❌ Won't match "reliable car for professionals"  

**Solution: Full PDF text + structured specs:**
```
Content: "Toyota Camry is a premium midsize sedan renowned for its 
reliability, spacious interiors, and smooth performance. It caters
to professionals, small families...

STRUCTURED SPECIFICATIONS
Model: Camry
Trim: LE
..."
```
✅ Matches semantic queries  
✅ Has structured metadata for filtering  
✅ Supports Notebook1's multi-query variants  


In [13]:
from langchain_core.documents import Document

def create_chunks_from_specs(
    pdf_path: Path,
    specs: List[KeySpec],
    original_text: str
) -> List[Document]:
    """
    Create hybrid chunks: Full PDF text + structured specs.
    One chunk per model/trim configuration.
    
    PDFs average 2,843 chars - well within embedding limits!
    Final chunks: ~3,500 chars (PDF + specs) = ~875 tokens.
    """
    chunks = []
    
    for i, spec in enumerate(specs):
        # Build structured specs section
        spec_lines = [f"Model: {spec.model}"]
        
        if spec.trim:
            spec_lines.append(f"Trim: {spec.trim}")
        
        if spec.mpg_combined:
            spec_lines.append(f"Fuel Economy: {spec.mpg_combined} MPG combined")
            if spec.mpg_city and spec.mpg_hwy:
                spec_lines.append(f"  ({spec.mpg_city} city / {spec.mpg_hwy} highway)")        
        if spec.ev_only_range_mi:
            spec_lines.append(f"EV-Only Range: {spec.ev_only_range_mi} miles")
        
        if spec.total_range_mi:
            spec_lines.append(f"Total Range: {spec.total_range_mi} miles")
        
        if spec.towing_max_lbs:
            spec_lines.append(f"Towing Capacity: {spec.towing_max_lbs} lbs")
        
        if spec.seats:
            spec_lines.append(f"Seating: {spec.seats} passengers")
        
        if spec.drivetrain:
            spec_lines.append(f"Drivetrain: {spec.drivetrain}")
        
        if spec.starting_price_mentions:
            spec_lines.append(f"Pricing: {', '.join(spec.starting_price_mentions)}")
        
        # HYBRID CONTENT: Full PDF text + structured specs
        page_content = f"""{original_text}

{'='*60}
STRUCTURED SPECIFICATIONS
{'='*60}

{chr(10).join(spec_lines)}"""        
        # Metadata: All KeySpec fields (ChromaDB compatible)
        metadata = {
            "source": pdf_path.name,
            "chunk_id": f"{pdf_path.stem}_{i:03d}",
            "model": spec.model,
        }
        
        # Add optional fields
        if spec.trim:
            metadata["trim"] = spec.trim
        if spec.mpg_city:
            metadata["mpg_city"] = spec.mpg_city
        if spec.mpg_hwy:
            metadata["mpg_hwy"] = spec.mpg_hwy
        if spec.mpg_combined:
            metadata["mpg_combined"] = spec.mpg_combined
        if spec.ev_only_range_mi:
            metadata["ev_only_range_mi"] = spec.ev_only_range_mi
        if spec.total_range_mi:
            metadata["total_range_mi"] = spec.total_range_mi
        if spec.towing_max_lbs:
            metadata["towing_max_lbs"] = spec.towing_max_lbs
        if spec.seats:
            metadata["seats"] = spec.seats
        if spec.drivetrain:
            metadata["drivetrain"] = spec.drivetrain
        
        # Convert list to string for ChromaDB
        if spec.starting_price_mentions:
            metadata["starting_price_mentions"] = ", ".join(spec.starting_price_mentions)
        
        chunks.append(Document(
            page_content=page_content,
            metadata=metadata
        ))
    
    return chunks

print("✓ Hybrid chunking function defined")


✓ Hybrid chunking function defined


## Section 9: ChromaDB Setup


In [14]:
from langchain_community.vectorstores import Chroma
from langchain_google_vertexai import VertexAIEmbeddings

# Initialize embeddings model
embeddings_model = VertexAIEmbeddings(model_name=EMBED_MODEL_ID)

print(f"✓ Embeddings model ready: {EMBED_MODEL_ID}")


✓ Embeddings model ready: text-embedding-005


In [15]:
# Initialize vectorstore
vectorstore = Chroma(
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIR,
    embedding_function=embeddings_model
)

print(f"✓ Vectorstore ready: {COLLECTION_NAME}")
print(f"  Persist directory: {PERSIST_DIR}")

# Check current size
try:
    current_count = vectorstore._collection.count()
    print(f"  Current documents: {current_count}")
except:
    print(f"  Collection is empty or new")


/var/folders/n4/66bq258d6xq6lscwjlgs6q500000gn/T/ipykernel_104/4248553841.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


✓ Vectorstore ready: toyota_specs
  Persist directory: ../data/chroma
  Current documents: 0


## Section 10: Batched Ingestion Pipeline

### Why Batch Processing?

- **Avoids rate limits**: Process 5 chunks, wait 2s, repeat
- **Better error handling**: Can retry individual batches
- **Progress tracking**: Shows which chunks are being embedded

### force_rebuild Parameter

- `True`: Clears all 123 old documents, rebuilds from scratch
- `False`: Adds to existing collection (may cause duplicates)


In [16]:
import time

def ingest_toyota_pdfs_batched(
    pdf_dir: str,
    vectorstore: Chroma,
    llm,
    force_rebuild: bool = False,
    batch_size: int = 5,
    delay_between_batches: float = 2.0
) -> Dict[str, Any]:
    """
    Batch ingestion with delays to avoid rate limits.
    
    Args:
        batch_size: Process N chunks, embed, then delay
        delay_between_batches: Seconds to wait between batches
    """
    if force_rebuild:
        print("🔄 Force rebuild: Clearing collection...")
        vectorstore.delete_collection()
        vectorstore = Chroma(
            collection_name=COLLECTION_NAME,
            persist_directory=PERSIST_DIR,
            embedding_function=embeddings_model
        )
        print("   ✓ Collection cleared")
    
    pdf_files = list_pdf_files(pdf_dir)
    print(f"\n📄 Found {len(pdf_files)} PDFs")
    print("="*60)
    
    all_chunks = []
    stats = {
        "total_pdfs": len(pdf_files),
        "total_chunks": 0,
        "pdfs_processed": [],
        "errors": []
    }
    
    for pdf_path in pdf_files:
        try:
            print(f"\n📖 Processing: {pdf_path.name}")
            
            # Extract specs AND original text
            specs, original_text = extract_specs_from_pdf_llm(pdf_path, llm)
            print(f"  ✓ Extracted {len(specs)} configurations")
            
            # Create hybrid chunks
            chunks = create_chunks_from_specs(pdf_path, specs, original_text)
            print(f"  ✓ Created {len(chunks)} chunks")
            
            all_chunks.extend(chunks)
            stats["pdfs_processed"].append({
                "filename": pdf_path.name,
                "configs": len(specs),
                "chunks": len(chunks)
            })
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
            stats["errors"].append({
                "file": pdf_path.name,
                "error": str(e)
            })
    
    # Add chunks in batches with delays
    if all_chunks:
        print(f"\n💾 Adding {len(all_chunks)} chunks in batches...")
        print("="*60)
        
        for i in range(0, len(all_chunks), batch_size):
            batch = all_chunks[i:i+batch_size]
            batch_num = i//batch_size + 1
            total_batches = (len(all_chunks) + batch_size - 1) // batch_size
            
            print(f"\n  Batch {batch_num}/{total_batches}: Adding {len(batch)} chunks...")
            vectorstore.add_documents(batch)
            print(f"  ✓ Batch {batch_num} embedded")
            
            if i + batch_size < len(all_chunks):
                print(f"  ⏳ Waiting {delay_between_batches}s before next batch...")
                time.sleep(delay_between_batches)
        
        print(f"\n✅ All {len(all_chunks)} chunks successfully added!")
    
    stats["total_chunks"] = len(all_chunks)
    return stats

print("✓ Batched ingestion function defined")


✓ Batched ingestion function defined


## Section 11: Run Ingestion

**⚠️ Set `force_rebuild=True` to clear old data!**

This will:
1. Delete 123 old documents
2. Process 8 PDFs
3. Extract ~29 configurations
4. Create 29 hybrid chunks
5. Embed in batches of 5 with 2s delays


In [17]:
ingestion_stats = ingest_toyota_pdfs_batched(
    pdf_dir=DATA_DIR,
    vectorstore=vectorstore,
    llm=chat_llm,
    force_rebuild=True,  # ⬅️ SET TO TRUE TO CLEAR OLD DATA
    batch_size=5,
    delay_between_batches=2.0
)

print("\n" + "="*60)
print("INGESTION COMPLETE")
print("="*60)
print(f"PDFs processed: {ingestion_stats['total_pdfs']}")
print(f"Total chunks: {ingestion_stats['total_chunks']}")
print(f"Successful: {len(ingestion_stats['pdfs_processed'])}")

if ingestion_stats['errors']:
    print(f"\n⚠️ Errors: {len(ingestion_stats['errors'])}")
    for err in ingestion_stats['errors']:
        print(f"  - {err['file']}: {err['error']}")
else:
    print("\n✅ No errors")


🔄 Force rebuild: Clearing collection...
   ✓ Collection cleared

📄 Found 8 PDFs

📖 Processing: Introduction_to_Toyota_Car_Sales.pdf
  ✓ Extracted 0 configurations
  ✓ Created 0 chunks

📖 Processing: Toyota_Camry_Specifications.pdf
  ✓ Extracted 4 configurations
  ✓ Created 4 chunks

📖 Processing: Toyota_Corolla_Specifications.pdf
  ✓ Extracted 5 configurations
  ✓ Created 5 chunks

📖 Processing: Toyota_Highlander_Specifications.pdf
  ✓ Extracted 7 configurations
  ✓ Created 7 chunks

📖 Processing: Toyota_Prius_Specifications.pdf
  ✓ Extracted 7 configurations
  ✓ Created 7 chunks

📖 Processing: Toyota_RAV4_Specifications.pdf
  ✓ Extracted 7 configurations
  ✓ Created 7 chunks

📖 Processing: Toyota_Tacoma_Specifications.pdf
  ✓ Extracted 3 configurations
  ✓ Created 3 chunks

📖 Processing: Toyota_bZ4X_Specifications.pdf
  ✓ Extracted 2 configurations
  ✓ Created 2 chunks

💾 Adding 35 chunks in batches...

  Batch 1/7: Adding 5 chunks...
  ✓ Batch 1 embedded
  ⏳ Waiting 2.0s before next 

### Reconnect to New Collection

**Why is this needed?**

When `force_rebuild=True`, the ingestion function:
1. Deletes the old collection
2. Creates a NEW collection with a NEW ID
3. But due to Python variable scoping, the global `vectorstore` variable still points to the OLD (deleted) collection

We need to reinitialize `vectorstore` to point to the new collection.


In [18]:
# ⚠️ IMPORTANT: Reconnect to collection after force_rebuild
# The ingestion function created a new collection with a new ID
# We need to reinitialize the vectorstore variable to point to it

vectorstore = Chroma(
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIR,
    embedding_function=embeddings_model
)

print("✓ Vectorstore reconnected to new collection")
try:
    doc_count = vectorstore._collection.count()
    print(f"  Documents in collection: {doc_count}")
except Exception as e:
    print(f"  Could not get count: {e}")


✓ Vectorstore reconnected to new collection
  Documents in collection: 35


## Section 12: Validation - Semantic Search

Test that natural language queries work. These queries should match chunks based on **descriptive text**, not just metadata.


In [19]:
print("="*60)
print("SEMANTIC SEARCH VALIDATION")
print("="*60)

semantic_queries = [
    "comfortable family sedan",
    "reliable car for professionals",
    "eco-friendly hybrid vehicle",
    "powerful off-road truck"
]

for query in semantic_queries:
    print(f"\nQuery: '{query}'")
    print("-"*60)
    results = vectorstore.similarity_search(query, k=3)
    
    for i, doc in enumerate(results, 1):
        model = doc.metadata.get('model', 'N/A')
        trim = doc.metadata.get('trim', '')
        text_len = len(doc.page_content)
        
        print(f"  {i}. {model} {trim}")
        print(f"     Content length: {text_len} chars")
        
        # Verify has semantic text
        has_semantic = any(word in doc.page_content.lower() 
                          for word in ['reliable', 'comfortable', 'efficient', 'family', 'professional'])
        print(f"     Has semantic text: {'✅' if has_semantic else '❌'}")
        
        if text_len > 100:
            preview = doc.page_content[:150].replace('\n', ' ')
            print(f"     Preview: {preview}...")

print("\n" + "="*60)
print("✅ Semantic search validation complete")


SEMANTIC SEARCH VALIDATION

Query: 'comfortable family sedan'
------------------------------------------------------------
  1. Corolla XSE
     Content length: 2893 chars
     Has semantic text: ✅
     Preview: rag/Toyota_Corolla_Specifications.md Toyota Corolla: A Best-Selling Sedan Overview The Toyota Corolla is one of the best-selling sedans worldwide, kno...
  2. Corolla SE
     Content length: 2875 chars
     Has semantic text: ✅
     Preview: rag/Toyota_Corolla_Specifications.md Toyota Corolla: A Best-Selling Sedan Overview The Toyota Corolla is one of the best-selling sedans worldwide, kno...
  3. Corolla LE
     Content length: 2892 chars
     Has semantic text: ✅
     Preview: rag/Toyota_Corolla_Specifications.md Toyota Corolla: A Best-Selling Sedan Overview The Toyota Corolla is one of the best-selling sedans worldwide, kno...

Query: 'reliable car for professionals'
------------------------------------------------------------
  1. Corolla XSE
     Content length: 2893 chars

## Section 13: Validation - Metadata Filtering

Test that KeySpec metadata filters work correctly.


In [20]:
print("="*60)
print("METADATA FILTERING VALIDATION")
print("="*60)

# Test 1: Filter by model
print("\n1. Filter: model='Camry'")
results = vectorstore.similarity_search("Toyota", k=10, filter={"model": "Camry"})
print(f"   Found {len(results)} Camry chunks")
if results:
    for doc in results[:3]:
        trim = doc.metadata.get('trim', 'N/A')
        print(f"     - Camry {trim}")

# Test 2: Filter by MPG
print("\n2. Filter: mpg_combined >= 30")
try:
    results = vectorstore.get(where={"mpg_combined": {"$gte": 30}})
    print(f"   Found {len(results['ids'])} high-efficiency chunks")
    if results['metadatas']:
        for meta in results['metadatas'][:5]:
            model = meta.get('model', 'N/A')
            trim = meta.get('trim', '')
            mpg = meta.get('mpg_combined', 'N/A')
            print(f"     - {model} {trim}: {mpg} MPG")
except Exception as e:
    print(f"   Error: {e}")

# Test 3: Filter by model AND trim
# Note: ChromaDB requires $and operator for multiple conditions
print("\n3. Filter: model='RAV4' AND trim='TRD Pro'")
results = vectorstore.similarity_search(
    "Toyota", 
    k=5, 
    filter={
        "$and": [
            {"model": "RAV4"},
            {"trim": "TRD Pro"}
        ]
    }
)
if results:
    print(f"   Found {len(results)} matching chunk(s)")
    for doc in results:
        print(f"     Metadata: {doc.metadata}")
else:
    print("   No matches found (check if this trim exists)")

print("\n" + "="*60)
print("✅ Metadata filtering validation complete")


METADATA FILTERING VALIDATION

1. Filter: model='Camry'
   Found 4 Camry chunks
     - Camry XSE V6
     - Camry SE
     - Camry Hybrid

2. Filter: mpg_combined >= 30
   Found 3 high-efficiency chunks
     - Prius Prime SE: 133.0 MPG
     - Prius Prime XSE: 133.0 MPG
     - Prius Prime XSE Premium: 133.0 MPG

3. Filter: model='RAV4' AND trim='TRD Pro'
   No matches found (check if this trim exists)

✅ Metadata filtering validation complete


## Section 14: Validation - Hybrid Query

Test combined semantic search + metadata filtering, simulating Notebook1's planning queries.


In [21]:
print("="*60)
print("HYBRID QUERY VALIDATION")
print("="*60)

# Simulate Notebook1 planning query
print("\nQuery: 'Which Toyota is comfortable for families under $30,000?'")
print("Approach: Semantic search + price filter (post-processing)")
print("-"*60)

# Step 1: Semantic search
results = vectorstore.similarity_search(
    "comfortable spacious family sedan reliable",
    k=10
)

print(f"\nSemantic search returned {len(results)} chunks")

# Step 2: Filter by price (post-processing for demo)
affordable = []
for doc in results:
    price_str = doc.metadata.get('starting_price_mentions', '')
    # Check for prices under $30,000
    if any(p in price_str for p in ['26,000', '27,000', '28,000', '29,000']):
        affordable.append(doc)

print(f"\n✓ Found {len(affordable)} affordable family-friendly options:")
print("-"*60)

for doc in affordable[:5]:  # Show top 5
    model = doc.metadata.get('model', 'N/A')
    trim = doc.metadata.get('trim', '')
    price = doc.metadata.get('starting_price_mentions', 'N/A')
    mpg = doc.metadata.get('mpg_combined', 'N/A')
    seats = doc.metadata.get('seats', 'N/A')
    
    print(f"\n  {model} {trim}")
    print(f"    Price: {price}")
    print(f"    MPG: {mpg}")
    print(f"    Seats: {seats}")

print("\n" + "="*60)
print("✅ Hybrid query validation complete")
print("\nThis demonstrates how Notebook1's planning queries will work!")


HYBRID QUERY VALIDATION

Query: 'Which Toyota is comfortable for families under $30,000?'
Approach: Semantic search + price filter (post-processing)
------------------------------------------------------------

Semantic search returned 10 chunks

✓ Found 4 affordable family-friendly options:
------------------------------------------------------------

  Corolla XSE
    Price: $27,000
    MPG: N/A
    Seats: N/A

  Camry Hybrid
    Price: $29,000
    MPG: N/A
    Seats: N/A

  Camry SE
    Price: $28,000
    MPG: N/A
    Seats: N/A

  Camry LE
    Price: $26,000
    MPG: N/A
    Seats: N/A

✅ Hybrid query validation complete

This demonstrates how Notebook1's planning queries will work!


## Section 15: Ingestion Summary

Final statistics and next steps.


In [22]:
print("="*60)
print("INGESTION SUMMARY")
print("="*60)

# Collection stats
collection = vectorstore._collection
total_docs = collection.count()
print(f"\nTotal documents in collection: {total_docs}")

# Sample chunk size
sample = vectorstore.get(limit=1)
if sample['documents']:
    sample_length = len(sample['documents'][0])
    print(f"Sample chunk size: {sample_length} chars (~{sample_length//4} tokens)")

# Metadata fields
if sample['metadatas']:
    print(f"\nMetadata fields available:")
    for key in sorted(sample['metadatas'][0].keys()):
        print(f"  - {key}")

# Processing summary
print(f"\nProcessing summary:")
for pdf_info in ingestion_stats['pdfs_processed']:
    print(f"  - {pdf_info['filename']}: {pdf_info['chunks']} chunks")

print("\n" + "="*60)
print("✅ INGESTION COMPLETE AND VALIDATED!")
print("="*60)
print("\nNext steps:")
print("  1. Run Notebook1_Prompt_to_Plan_Llama_ORDERED.ipynb")
print("  2. Test planning queries with semantic search + metadata filtering")
print("  3. Verify multi-query variants work with rich chunk content")
print("\nChunk structure:")
print("  ✅ Full PDF text for semantic search")
print("  ✅ Structured specs for readability")
print("  ✅ KeySpec metadata for precise filtering")
print("  ✅ Supports ALL Notebook1 planning concepts")


INGESTION SUMMARY

Total documents in collection: 35
Sample chunk size: 3278 chars (~819 tokens)

Metadata fields available:
  - chunk_id
  - drivetrain
  - model
  - mpg_city
  - mpg_hwy
  - source
  - starting_price_mentions
  - trim

Processing summary:
  - Introduction_to_Toyota_Car_Sales.pdf: 0 chunks
  - Toyota_Camry_Specifications.pdf: 4 chunks
  - Toyota_Corolla_Specifications.pdf: 5 chunks
  - Toyota_Highlander_Specifications.pdf: 7 chunks
  - Toyota_Prius_Specifications.pdf: 7 chunks
  - Toyota_RAV4_Specifications.pdf: 7 chunks
  - Toyota_Tacoma_Specifications.pdf: 3 chunks
  - Toyota_bZ4X_Specifications.pdf: 2 chunks

✅ INGESTION COMPLETE AND VALIDATED!

Next steps:
  1. Run Notebook1_Prompt_to_Plan_Llama_ORDERED.ipynb
  2. Test planning queries with semantic search + metadata filtering
  3. Verify multi-query variants work with rich chunk content

Chunk structure:
  ✅ Full PDF text for semantic search
  ✅ Structured specs for readability
  ✅ KeySpec metadata for precise fil